pip install nest_asyncio

git clone --depth=1 https://github.com/twintproject/twint.git
cd twint
pip3 install . -r requirements.txt

+ aiohttp==3.7.0 setzen in requirements.txt

In [1]:
import twint
import pandas as pd
import datetime
import schedule
import time
import os
import nest_asyncio
nest_asyncio.apply()
pd.set_option('display.max_columns', None)

c = twint.Config()

In [2]:
def searchTwint(keyword, since=None, until=None, language=None, limit=None, createDataFrame=True, hide_out=True):
    c = twint.Config()
    c.Search = "{} lang:de".format(keyword)
    c.Filter_retweets = False
    if language is not None:
        c.Lang = language
    if since is not None:
        c.Since = since
    if until is not None:
        c.Until = until
    if limit is not None:
        c.Limit = limit
    c.Hide_output = hide_out
    if createDataFrame:
        c.Pandas = True
    twint.run.Search(c)
    if createDataFrame:
        Tweets_df = twint.storage.panda.Tweets_df
        return Tweets_df

In [3]:
df = searchTwint("corona", since="2022-05-18", until="2022-05-19", language=None, limit=500, createDataFrame=True)

In [4]:
df.head(4)

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1527075828277952515,1526948275726729217,1.652918e+12,2022-05-19 01:57:03,+0200,,@WuChenFu3 @DrNathanCole Manchmal ist er ganz ...,de,[],[],1238892997535825920,1238892997535825920,HN_112_,🇺🇦 Polonaise mit Abstand & mit enthemmten Mob 🧹,4,01,https://twitter.com/HN_112_/status/15270758282...,[],[https://pbs.twimg.com/media/FTFDX1mWIAUh1Ro.jpg],1,https://pbs.twimg.com/media/FTFDX1mWIAUh1Ro.jpg,False,1,0,0,,corona lang:de,,,,,,,"[{'screen_name': 'WuChenFu3', 'name': 'WuChenF...",,,,
1,1527075720224284672,1527075720224284672,1.652918e+12,2022-05-19 01:56:38,+0200,,New post (Corona: Warum sich manche noch nicht...,de,[],[],43948964,43948964,ATGBrokers,Alexander,4,01,https://twitter.com/ATGBrokers/status/15270757...,"[http://my24group.com, https://my24group.com/c...",[],0,,False,0,0,0,,corona lang:de,,,,,,,[],,,,
2,1527075580121862145,1527075580121862145,1.652918e+12,2022-05-19 01:56:04,+0200,,Corona Pandemiestufe 3 - wie viele Haushalte d...,de,[],[],705825629120405509,705825629120405509,malnefrage,malnefrage.de,4,01,https://twitter.com/malnefrage/status/15270755...,[https://www.malnefrage.de/frage/corona-pandem...,[],0,,False,0,0,0,,corona lang:de,,,,,,,[],,,,
3,1527075577852747778,1527075577852747778,1.652918e+12,2022-05-19 01:56:04,+0200,,Sinnlose PCR-Tests: ➡️ Australische Behörde g...,de,"[covid19, sarscov2, corona, pcrtests]",[],284182005,284182005,Roellchen2011,Susanne*Roellchen,4,01,https://twitter.com/Roellchen2011/status/15270...,[https://report24.news/sinnlose-pcr-tests-aust...,[],0,,False,2,0,2,,corona lang:de,,,,,,,[],,,,


In [6]:
# Duplikate werden entfernt
df = df.drop_duplicates(subset=["id"])

In [7]:
df["date"] = df["date"].astype("datetime64")
#df["created_at"] = df["created_at"].astype("datetime64")
df = df[df["language"] == "de"]
df["Wochentag"] = df["date"].apply(lambda x: x.weekday())
#df["Stunde"] = df["time"].apply(lambda x: x.hour)
df["Originale_Tweetlaenge"] = df["tweet"].apply(lambda x: len(x))

In [8]:
df

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Wochentag,Originale_Tweetlaenge
0,1527075828277952515,1526948275726729217,1.652918e+12,2022-05-19 01:57:03,+0200,,@WuChenFu3 @DrNathanCole Manchmal ist er ganz ...,de,[],[],1238892997535825920,1238892997535825920,HN_112_,🇺🇦 Polonaise mit Abstand & mit enthemmten Mob 🧹,4,01,https://twitter.com/HN_112_/status/15270758282...,[],[https://pbs.twimg.com/media/FTFDX1mWIAUh1Ro.jpg],1,https://pbs.twimg.com/media/FTFDX1mWIAUh1Ro.jpg,False,1,0,0,,corona lang:de,,,,,,,"[{'screen_name': 'WuChenFu3', 'name': 'WuChenF...",,,,,3,258
1,1527075720224284672,1527075720224284672,1.652918e+12,2022-05-19 01:56:38,+0200,,New post (Corona: Warum sich manche noch nicht...,de,[],[],43948964,43948964,ATGBrokers,Alexander,4,01,https://twitter.com/ATGBrokers/status/15270757...,"[http://my24group.com, https://my24group.com/c...",[],0,,False,0,0,0,,corona lang:de,,,,,,,[],,,,,3,138
2,1527075580121862145,1527075580121862145,1.652918e+12,2022-05-19 01:56:04,+0200,,Corona Pandemiestufe 3 - wie viele Haushalte d...,de,[],[],705825629120405509,705825629120405509,malnefrage,malnefrage.de,4,01,https://twitter.com/malnefrage/status/15270755...,[https://www.malnefrage.de/frage/corona-pandem...,[],0,,False,0,0,0,,corona lang:de,,,,,,,[],,,,,3,204
3,1527075577852747778,1527075577852747778,1.652918e+12,2022-05-19 01:56:04,+0200,,Sinnlose PCR-Tests: ➡️ Australische Behörde g...,de,"[covid19, sarscov2, corona, pcrtests]",[],284182005,284182005,Roellchen2011,Susanne*Roellchen,4,01,https://twitter.com/Roellchen2011/status/15270...,[https://report24.news/sinnlose-pcr-tests-aust...,[],0,,False,2,0,2,,corona lang:de,,,,,,,[],,,,,3,143
4,1527075324516831232,1526868523825475584,1.652918e+12,2022-05-19 01:55:03,+0200,,@womawo @ABaerbock @Minister1053 Hören Sie auf...,de,[],[],1451534855242719232,1451534855242719232,Palindromaviva,Palindrom,4,01,https://twitter.com/Palindromaviva/status/1527...,[],[],0,,False,2,0,0,,corona lang:de,,,,,,,"[{'screen_name': 'womawo', 'name': '🇺🇦 Marita'...",,,,,3,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1527021374669303808,1526856168987041792,1.652905e+12,2022-05-18 22:20:41,+0200,,@altesFlederweib Auch ich verarbeite Missbrauc...,de,[],[],2347422328,2347422328,EsunaEspina,Poemsforyoureyes,3,22,https://twitter.com/EsunaEspina/status/1527021...,[],[],0,,False,1,0,0,,corona lang:de,,,,,,,"[{'screen_name': 'altesFlederweib', 'name': 'ꋬ...",,,,,2,241
496,1527021313608626178,1527021313608626178,1.652905e+12,2022-05-18 22:20:26,+0200,,".@OlafScholz Sehr geehrter Herr Scholz, BM La...",de,[lauterbachruecktrittjetzt],[],1486437942168109056,1486437942168109056,ONose63,OrangeNose63,3,22,https://twitter.com/ONose63/status/15270213136...,[],[],0,,False,1,0,0,,corona lang:de,,,,,,,[],,,,,2,275
497,1527021308432814081,1527016716395499520,1.652905e+12,2022-05-18 22:20:25,+0200,,@nachenle okay yes muss schauen wie es mir bis...,de,[],[],4795349498,4795349498,hyunjinhoon,jana ♠️ misses enha,3,22,https://twitter.com/hyunjinhoon/status/1527021...,[],[],0,,False,0,1,0,,corona lang:de,,,,,,,"[{'screen_name': 'nachenle', 'name': 'nati', '...",,,,,2,99
498,1527021291244486666,1526643189423161345,1.652905e+12,2022-05-18 22:20:21,+0200,,@JSchmitzLeipzig Dann muss die Angst allerding...,de,"[panikeltern, corona]",[],1482262001095258115,1482262001095258115,Andreas_Georg_,Andreas,3,22,https://twitter.com/Andreas_Georg_/status/1527...,[],[],0,,False,6,0,0,,corona lang:de,,,,,,,"[{'screen_name': 'JSchmitzLeipzig', 'name': 'J...",,,,,2,294
